<a href="https://colab.research.google.com/github/LavanyaMQuantumThinker/Quantum-/blob/main/Quantum_penneylane_irisdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pennylane scikit-learn numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 104.1 MB/s eta 0:00:00


In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [8]:
# Load data (use only 2 classes for binary classification)
iris = load_iris()
X = iris.data[iris.target != 2]
y = iris.target[iris.target != 2]

# Normalize and split
X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
#Define Quantum Circuit

import pennylane as qml
from pennylane import numpy as pnp

n_qubits = 4
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev, interface="torch")
def quantum_circuit(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return qml.expval(qml.PauliZ(0))


In [4]:
#Create Hybrid Model

from pennylane.qnn import TorchLayer
import torch
import torch.nn as nn

# Define QLayer
weight_shapes = {"weights": (3, n_qubits)}  # 3 layers of entanglement
qlayer = TorchLayer(quantum_circuit, weight_shapes)

# Combine with classical layer
class HybridModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4, 4)
        self.qlayer = qlayer
        self.fc2 = nn.Linear(1, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.qlayer(x)
        x = self.fc2(x)
        return torch.sigmoid(x)


In [12]:
print(X_train_torch.shape)  # Should be (N, 4)
print(y_train_torch.shape)  # Should be (N, 1)


torch.Size([80, 4])
torch.Size([80, 1])


In [16]:
model = HybridModel()
opt = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.BCELoss()

# Prepare data
X_train_torch = torch.tensor(X_train, dtype=torch.float32).view(-1,4)
y_train_torch = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)

# Training loop
for epoch in range(20):  # Keep small due to quantum simulation slowness
    y_pred = model(X_train_torch)
    loss = loss_fn(y_pred, y_train_torch)
    opt.zero_grad()
    loss.backward()
    opt.step()
    print(f"Epoch {epoch+1}: loss = {loss.item():.4f}")


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x80 and 1x1)